In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle 
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, LassoCV
from sklearn import metrics
from sys import platform
import scipy as sp
import scipy.io
from scipy.stats import pearsonr
import os
from random import randrange
from itertools import combinations

In [ ]:
# Phase 1: load individual connectivity matrices, create group similarity matrices, calculate success rate and chance level

In [ ]:
# load individual mats, take lower triangle without diagonal, concatenate all subs together to create group matrices per timepoint
def mat2vec(datadir,subs,length):
    mats=[x for x in os.listdir(datadir) if x[:length] in subs]
    mats.sort()
    size_mat=scipy.io.loadmat(datadir + '/' + mats[1])
    a=list(size_mat.keys())
    size_mat=size_mat[a[3]]
    i=0
    dim=np.shape(size_mat)
    N=len(mats)
    bigmat=np.zeros([int((dim[0]*(dim[0]-1))/2),N])
    for file in mats:
        # print(file)
        submat=scipy.io.loadmat(datadir + '/' + file)
        b=list(submat.keys())
        submat=submat[b[3]]
        bigmat[:,i]=submat[np.tril_indices(dim[1], k = -1)]
        i+=1
    return bigmat

In [ ]:
# create similarity matrices for two timepoints 
def create_cor_mat2(X,Y):
    sub_cor=np.corrcoef(X.T, Y.T)[:np.shape(X)[1], np.shape(X)[1]:]
    sub_cor=pd.DataFrame(sub_cor.T)
    return sub_cor

In [ ]:
# calculate success: is diagonal value the highest value in the row
def calc_success(sub_cor):
    counter = 0
    for t in range(0,np.shape(sub_cor)[1]):
        val = sub_cor.loc[t,t]
        max_val=max(sub_cor.loc[:,t])
        if max_val==val:
            counter+=1
    success_rate=counter/(np.shape(sub_cor)[1])
    return counter,success_rate

In [ ]:
# Calculate chance rate
def perm(sub_cor, K):
    perm_success_rate=[]
    for i in range(K):
        shuffled_cor_mat=sub_cor.sample(frac=1).reset_index(drop=True)
        shuffled_counter,shuffled_sucess_rate=calc_success(shuffled_cor_mat)
        perm_success_rate.append(shuffled_sucess_rate)
    chance_level=np.mean(perm_success_rate)
    return chance_level, perm_success_rate

In [ ]:
# Soldiers

In [ ]:
rootdir1=('/Volumes/homes/Noga/full_data/Results/T1')
subs1=[x for x in os.listdir(rootdir1) if len(x)==4]
rootdir2=('/Volumes/homes/Noga/full_data/Results/T2')
subs2=[x for x in os.listdir(rootdir2) if len(x)==4]
rootdir3=('/Volumes/homes/Noga/full_data/Results/T3')
subs3=[x for x in os.listdir(rootdir3) if len(x)==4]
rootdir4=('/Volumes/homes/Noga/full_data/Results/T4')
subs4=[x for x in os.listdir(rootdir4) if len(x)==4]
subs4=np.intersect1d(subs1,subs4)

subs=np.intersect1d(np.intersect1d(np.intersect1d(subs1,subs4),subs2),subs3)
atlas='Schaefer1000'

In [ ]:
mat1=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/' + atlas,subs2,4)
mat2=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T2/' + atlas,subs2,4)
sub_cor12=create_cor_mat2(mat1,mat2)
counter12,success_rate12 = calc_success(sub_cor12)
chance_level12, perm_success_rate12=perm(sub_cor12, 1000)
print('Number of Participants: %s\nSucessful Fingerprinting: %s\nSuccess Rate: %s\nChance-Level Success rate: %s' % (np.shape(sub_cor12)[1],counter12,success_rate12,chance_level12))

In [ ]:
mat1=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/' + atlas,subs3,4)
mat3=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T3/' + atlas,subs3,4)
sub_cor13=create_cor_mat2(mat1,mat3)
counter13,success_rate13 = calc_success(sub_cor13)
chance_level13, perm_success_rate13=perm(sub_cor13, 1000)
print('Number of Participants: %s\nSucessful Fingerprinting: %s\nSuccess Rate: %s\nChance-Level Success rate: %s' % (np.shape(sub_cor13)[1],counter13,success_rate13,chance_level13))

In [ ]:
mat1=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/' + atlas,subs4,4)
mat4=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T4/' + atlas,subs4,4)
sub_cor14=create_cor_mat2(mat1,mat4)
counter14,success_rate14 = calc_success(sub_cor14)
chance_level14, perm_success_rate14=perm(sub_cor14, 1000)
print('Number of Participants: %s\nSucessful Fingerprinting: %s\nSuccess Rate: %s\nChance-Level Success rate: %s' % (np.shape(sub_cor14)[1],counter14,success_rate14,chance_level14))

In [ ]:
# Controls

In [ ]:
rootdir1_con=('/Volumes/homes/Noga/full_data/Results/Control/T1')
subs1_con=[x for x in os.listdir(rootdir1_con) if len(x)==3]
rootdir2_con=('/Volumes/homes/Noga/full_data/Results/Control/T2')
subs2_con=[x for x in os.listdir(rootdir2_con) if len(x)==3] #and x!='133']
rootdir3_con=('/Volumes/homes/Noga/full_data/Results/Control/T3')
subs3_con=[x for x in os.listdir(rootdir3_con) if len(x)==3]
rootdir4_con=('/Volumes/homes/Noga/full_data/Results/Control/T4')
subs4_con=[x for x in os.listdir(rootdir4_con) if len(x)==3]

subs_con=np.intersect1d(np.intersect1d(subs1_con,subs2_con),subs3_con)
len(subs_con)

In [ ]:
mat1_con=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T1/' + atlas,subs_con,3)
mat2_con=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T2/' + atlas,subs_con,3)
sub_cor12_con=create_cor_mat2(mat1_con,mat2_con)
counter12_con,success_rate12_con = calc_success(sub_cor12_con)
chance_level12_con, perm_success_rate12_con=perm(sub_cor12_con, 1000)
print('Number of Participants: %s\nSucessful Fingerprinting: %s\nSuccess Rate: %s\nChance-Level Success rate: %s' % (np.shape(sub_cor12_con)[1],counter12_con,success_rate12_con,chance_level12_con))

In [ ]:
mat1_con=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T1/' + atlas,subs_con,3)
mat3_con=mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T3/' + atlas,subs_con,3)
sub_cor13_con=create_cor_mat2(mat1_con,mat3_con)
counter13_con,success_rate13_con = calc_success(sub_cor13_con)
chance_level13_con, perm_success_rate13_con=perm(sub_cor13_con, 1000)
print('Number of Participants: %s\nSucessful Fingerprinting: %s\nSuccess Rate: %s\nChance-Level Success rate: %s' % (np.shape(sub_cor13_con)[1],counter13_con,success_rate13_con,chance_level13_con))

In [ ]:
# Phase 2: Network Feature Importance (AKA network contribution score)

In [ ]:
parcel_id=pd.read_excel('/Volumes/homes/Noga/Atlases/schaefer_parcellation/map1000to100.xlsx')
parcel_id
atlas='Schaefer1000'
networks=np.unique(parcel_id['Network'])

In [ ]:
def mat2vec_drop_multiple_networks(datadir,subs,length,networks):
    mats=[x for x in os.listdir(datadir) if x[:length] in subs]
    mats.sort()
    N=len(mats)
    bigmat=pd.DataFrame()

    network_index = parcel_id['Parcel_ID'].loc[parcel_id['Network'].isin(networks)]
    network_index=network_index-1
    for i, file in enumerate(mats):
        submat=scipy.io.loadmat(datadir + '/' + file)
        b=list(submat.keys())
        submat=submat[b[3]]
        submat=pd.DataFrame(submat)
        filtered_submat = submat.drop(network_index, axis=0)
        filtered_submat = filtered_submat.drop(network_index, axis=1)    
        dim=np.shape(filtered_submat)
        filtered_submat=np.array(filtered_submat)
        bigmat.loc[:,i]=filtered_submat[np.tril_indices(dim[1], k = -1)]

    bigmat=np.array(bigmat)
    return bigmat

In [ ]:
# Create a dictionary to store the success rates for each network for each iteration
network_iteration_success_without = {network: [] for network in networks}
network_iteration_success_with = {network: [] for network in networks}

# Iterate over all combinations of the networks
for r in range(1, len(networks)):
    for combo in combinations(networks, r):
        # Generate matrices and calculate success rates for the current combination
        mat1 = mat2vec_drop_multiple_networks('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/' + atlas, subs, 4, combo)
        mat2 = mat2vec_drop_multiple_networks ('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T2/' + atlas, subs, 4, combo)
        sub_cor = create_cor_mat2(mat1, mat2)
        counter, success_rate = calc_success(sub_cor)
        
       # Update network_iteration_success dictionary based on the current combination
        for network in networks:
            if network not in combo:
                network_iteration_success_without[network].append(success_rate)

        for network in networks:
            if network in combo:
                network_iteration_success_with[network].append(success_rate)


average_success_rates_without = {}
for network, success_rates in network_iteration_success_without.items():
    average_success_rates_without[network] = sum(success_rates) / len(success_rates)

average_success_rates_with = {}
for network, success_rates in network_iteration_success_with.items():
    average_success_rates_with[network] = sum(success_rates) / len(success_rates)

# Now average_success_rates dictionary contains the average success rate for each network
print(average_success_rates_without)
print(average_success_rates_with)


In [ ]:
# Create a dictionary to store the success rates for each network for each iteration
network_iteration_success_without = {network: [] for network in networks}
network_iteration_success_with = {network: [] for network in networks}

# Iterate over all combinations of the networks
for r in range(1, len(networks)):
    for combo in combinations(networks, r):
        # Generate matrices and calculate success rates for the current combination
        mat1 = mat2vec_drop_multiple_networks('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/' + atlas, subs, 4, combo)
        mat2 = mat2vec_drop_multiple_networks ('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T3/' + atlas, subs, 4, combo)
        sub_cor = create_cor_mat2(mat1, mat2)
        counter, success_rate = calc_success(sub_cor)
        
        # Update network_iteration_success dictionary based on the current combination
        for network in networks:
            if network not in combo:
                network_iteration_success_without[network].append(success_rate)

        for network in networks:
            if network in combo:
                network_iteration_success_with[network].append(success_rate)


average_success_rates_without = {}
for network, success_rates in network_iteration_success_without.items():
    average_success_rates_without[network] = sum(success_rates) / len(success_rates)

average_success_rates_with = {}
for network, success_rates in network_iteration_success_with.items():
    average_success_rates_with[network] = sum(success_rates) / len(success_rates)

# Now average_success_rates dictionary contains the average success rate for each network
print(average_success_rates_without)
print(average_success_rates_with)


In [ ]:
# Create a dictionary to store the success rates for each network for each iteration
network_iteration_success_without = {network: [] for network in networks}
network_iteration_success_with = {network: [] for network in networks}

# Iterate over all combinations of the networks
for r in range(1, len(networks)):
    for combo in combinations(networks, r):
        # Generate matrices and calculate success rates for the current combination
        mat1 = mat2vec_drop_multiple_networks('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/' + atlas, subs, 4, combo)
        mat2 = mat2vec_drop_multiple_networks ('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T4/' + atlas, subs, 4, combo)
        sub_cor = create_cor_mat(mat1, mat2)
        counter, success_rate = calc_success(sub_cor)
        
        # Update network_iteration_success dictionary based on the current combination
        for network in networks:
            if network not in combo:
                network_iteration_success_without[network].append(success_rate)

        for network in networks:
            if network in combo:
                network_iteration_success_with[network].append(success_rate)


average_success_rates_without = {}
for network, success_rates in network_iteration_success_without.items():
    average_success_rates_without[network] = sum(success_rates) / len(success_rates)

average_success_rates_with = {}
for network, success_rates in network_iteration_success_with.items():
    average_success_rates_with[network] = sum(success_rates) / len(success_rates)

# Now average_success_rates dictionary contains the average success rate for each network
print(average_success_rates_without)
print(average_success_rates_with)


In [ ]:
# Controls
rootdir1=('/Volumes/homes/Noga/full_data/Results/Control/T1/')
subs1=[x for x in os.listdir(rootdir1) if len(x)==3]
rootdir2=('/Volumes/homes/Noga/full_data/Results/Control/T2')
subs2=[x for x in os.listdir(rootdir2) if len(x)==3]
rootdir3=('/Volumes/homes/Noga/full_data/Results/Control/T3')
subs3=[x for x in os.listdir(rootdir3) if len(x)==3]

subs=np.intersect1d(np.intersect1d(subs1,subs2),subs3)

In [ ]:
# Create a dictionary to store the success rates for each network for each iteration
network_iteration_success_without = {network: [] for network in networks}
network_iteration_success_with = {network: [] for network in networks}

# Iterate over all combinations of the networks
for r in range(1, len(networks)):
    for combo in combinations(networks, r):
        # Generate matrices and calculate success rates for the current combination
        mat1 = mat2vec_drop_multiple_networks('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T1/' + atlas, subs, 3, combo)
        mat2 = mat2vec_drop_multiple_networks ('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T2/' + atlas, subs, 3, combo)
        sub_cor = create_cor_mat2(mat1, mat2)
        counter, success_rate = calc_success(sub_cor)
        
        # Update network_iteration_success dictionary based on the current combination
        for network in networks:
            if network not in combo:
                network_iteration_success_without[network].append(success_rate)

        for network in networks:
            if network in combo:
                network_iteration_success_with[network].append(success_rate)


average_success_rates_without = {}
for network, success_rates in network_iteration_success_without.items():
    average_success_rates_without[network] = sum(success_rates) / len(success_rates)

average_success_rates_with = {}
for network, success_rates in network_iteration_success_with.items():
    average_success_rates_with[network] = sum(success_rates) / len(success_rates)

# Now average_success_rates dictionary contains the average success rate for each network
print(average_success_rates_without)
print(average_success_rates_with)


In [ ]:
# Create a dictionary to store the success rates for each network for each iteration
network_iteration_success_without = {network: [] for network in networks}
network_iteration_success_with = {network: [] for network in networks}

# Iterate over all combinations of the networks
for r in range(1, len(networks)):
    for combo in combinations(networks, r):
        # Generate matrices and calculate success rates for the current combination
        mat1 = mat2vec_drop_multiple_networks('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T1/' + atlas, subs, 3, combo)
        mat2 = mat2vec_drop_multiple_networks ('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T3/' + atlas, subs, 3, combo)
        sub_cor = create_cor_mat2(mat1, mat2)
        counter, success_rate = calc_success(sub_cor)
        
        # Update network_iteration_success dictionary based on the current combination
        for network in networks:
            if network not in combo:
                network_iteration_success_without[network].append(success_rate)

        for network in networks:
            if network in combo:
                network_iteration_success_with[network].append(success_rate)


average_success_rates_without = {}
for network, success_rates in network_iteration_success_without.items():
    average_success_rates_without[network] = sum(success_rates) / len(success_rates)

average_success_rates_with = {}
for network, success_rates in network_iteration_success_with.items():
    average_success_rates_with[network] = sum(success_rates) / len(success_rates)

# Now average_success_rates dictionary contains the average success rate for each network
print(average_success_rates_without)
print(average_success_rates_with)


In [ ]:
# Phase 3: Permutation Test to explore group differences within timepoint

In [ ]:
# 1. shuffle between groups, create null distribution of success rate differences
# how to execute: 
# a. shuffle mat1-mat2 and mat1_con-mat2_con (shuffle couples)
# b. calc success and store in df
# c. create distrubution of success difference

def shuffle_matrices(s1,s2,c1,c2,K):
    perm_success_rate_diff=[]
    t1=pd.concat([s1,c1],axis=1)
    t2=pd.concat([s2,c2],axis=1)
    vector = [1] * np.shape(s1)[1] + [0] * np.shape(c1)[1]

    for _ in range(K):
        random.shuffle(vector)
        ind_s = [idx for idx, val in enumerate(vector) if val == 1]
        ind_c = [idx for idx, val in enumerate(vector) if val == 0]

        s1=t1.iloc[:,ind_s].reset_index(drop=True)
        s2=t2.iloc[:,ind_s].reset_index(drop=True)

        c1=t1.iloc[:,ind_c].reset_index(drop=True)
        c2=t2.iloc[:,ind_c].reset_index(drop=True)

        s_cor=create_cor_mat2(np.array(s1),np.array(s2))
        _,success_rate_s = calc_success(s_cor)

        c_cor=create_cor_mat2(np.array(c1),np.array(c2))
        _,success_rate_c = calc_success(c_cor)
        perm_success_rate_diff.append(success_rate_s-success_rate_c)
    return perm_success_rate_diff

In [ ]:
mat1=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/Schaefer1000',subs2,4))
mat2=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T2/Schaefer1000',subs2,4))
mat1_con=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T1/Schaefer1000',subs2_con,3))
mat2_con=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T2/Schaefer1000',subs2_con,3))

perm_success_rate_diff_12 = shuffle_matrices(mat1,mat2,mat1_con,mat2_con,1000)

In [ ]:
a=np.array(perm_success_rate_diff_12)
b=success_rate12-success_rate12_con
c=np.append(a,b)
print(len(c[c<=b])/len(c))
plt.hist(c,bins=10)

In [ ]:
mat1=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T1/Schaefer1000',subs3,4))
mat3=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/T3/Schaefer1000',subs3,4))
mat1_con=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T1/Schaefer1000',subs3_con,3))
mat3_con=pd.DataFrame(mat2vec('/Volumes/homes/Noga/full_data/Results/Atlas_CMs/Control/T3/Schaefer1000',subs3_con,3))

perm_success_rate_diff_13 = shuffle_matrices(mat1,mat3,mat1_con,mat3_con,1000)

In [ ]:
a=np.array(perm_success_rate_diff_13)
b=success_rate13-success_rate13_con
c=np.append(a,b)
print(len(c[c<=b])/len(c))
plt.hist(c,bins=10)

In [ ]:
# Phase 4: Calculate Stability, Uniqueness and Discriminitability

In [ ]:
def idiff(z_mat):
    idiff=[]
    iself_mat=[]
    iothers_mat=[]
    for i in range(np.shape(z_mat)[1]):
        n=np.shape(z_mat)[1]
        iself=z_mat.loc[i,i]
        row=z_mat.loc[i,:]
        row=row.drop(i).reset_index(drop=True)
        col=z_mat.loc[:,i]
        col=col.drop(i).reset_index(drop=True)
        iothers=1-((1/(2*(n-1)))*sum(row+col))
        iothers_for_disc_calc=((1/(2*(n-1)))*sum(row+col))
        idiff.append(iself/iothers_for_disc_calc)
        iself_mat.append(iself)
        iothers_mat.append(iothers)
    return idiff, iothers_mat, iself_mat

In [ ]:
idiff12, iothers12, iself12 =idiff(sub_cor12)
idiff13, iothers13, iself13=idiff(sub_cor13)
idiff14, iothers14, iself14=idiff(sub_cor14)
idiff12_con, iothers12_con, iself12_con =idiff(sub_cor12_con)
idiff13_con, iothers13_con, iself13_con=idiff(sub_cor13_con)

In [ ]:
t12_df=pd.DataFrame(columns=['id','Group','time','discriminability','iothers','iself'])
n=np.shape(sub_cor12)[1]
ids=[int(i) for i in subs2]
ids.sort()
t12_df.id=ids
t12_df.Group='Soldiers'
t12_df.time='Time-points 1&2'
t12_df.discriminability=idiff12
t12_df.iothers=iothers12
t12_df.iself=iself12

t13_df=pd.DataFrame(columns=['id','Group','time','discriminability','iothers','iself'])
n=np.shape(sub_cor13)[1]
ids=[int(i) for i in subs3]
ids.sort()
t13_df.id=ids
t13_df.Group='Soldiers'
t13_df.time='Time-points 1&3'
t13_df.discriminability=idiff13
t13_df.iothers=iothers13
t13_df.iself=iself13

t14_df=pd.DataFrame(columns=['id','Group','time','discriminability','iothers','iself'])
n=np.shape(sub_cor14)[1]
ids=[int(i) for i in subs4]
ids.sort()
t14_df.id=ids
t14_df.Group='Soldiers'
t14_df.time='Time-points 1&4'
t14_df.discriminability=idiff14
t14_df.iothers=iothers14
t14_df.iself=iself14

In [ ]:
t12_df_con=pd.DataFrame(columns=['id','Group','time','discriminability','iothers','iself'])
n=np.shape(sub_cor12_con)[1]
ids=[int(i) for i in subs2_con]
ids.sort()
t12_df_con.id=ids
t12_df_con.Group='Students'
t12_df_con.time='Time-points 1&2'
t12_df_con.discriminability=idiff12_con
t12_df_con.iothers=iothers12_con
t12_df_con.iself=iself12_con

t13_df_con=pd.DataFrame(columns=['id','Group','time','discriminability','iothers','iself'])
n=np.shape(sub_cor13_con)[1]
ids=[int(i) for i in subs3_con]
ids.sort()
t13_df_con.id=ids
t13_df_con.Group='Students'
t13_df_con.time='Time-points 1&3'
t13_df_con.discriminability=idiff13_con
t13_df_con.iothers=iothers13_con
t13_df_con.iself=iself13_con